In [1]:
import visual_behavior.validation.sdk as sdk_validation
from visual_behavior.validation.sdk import ValidateSDK
from visual_behavior import database as db
# import visual_behavior.visualization.qc.data_loading as dl
import visual_behavior.data_access.loading as dl
import visual_behavior.utilities as vbu
import h5py
%widescreen

/home/dougo/Code/AllenSDK/allensdk/brain_observatory/behavior/behavior_ophys_api/behavior_ophys_nwb_api.py:9: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
conn = db.Database('visual_behavior_data')
res = conn['sdk_validation']['validation_results'].find({'eye_tracking':0})
eye_tracking_failures = pd.DataFrame(list(res))
conn.close()

In [3]:
len(eye_tracking_failures)

248

In [4]:
def get_workflow_state(bsid):
    query = '''
    select oe.workflow_state
    from behavior_sessions bs
    join ophys_experiments oe on oe.ophys_session_id = bs.ophys_session_id
    where bs.id = {}
    '''.format(bsid)
    ans = db.lims_query(query)
    if isinstance(ans, pd.DataFrame):
        return 'passed' if (ans['workflow_state'] == 'passed').any() else 'failed'
    else:
        return ans
    
def get_path(well_known_files, name):
    '''
    get path from well_known_files dataframe
    inputs:
        well_known_files (dataframe with index set to name)
        name (string with desired name)
    returns:
        string with full filepath
    '''
    if name in well_known_files.index:
        return ''.join(well_known_files.loc[name][['storage_directory', 'filename']].tolist())
    else:
        return None
    
def get_movie_frame_count(ophys_session_id,movie_type='EyeTracking'):
    wkf = db.get_well_known_files(ophys_session_id)
    movie = vbu.Movie(get_path(wkf,'Raw{}Video'.format(movie_type)))
    return movie.frame_count

def get_videomon_frame_count(ophys_session_id,movie_type='EyeTracking'):
    wkf = db.get_well_known_files(ophys_session_id)
    movie_path = get_path(wkf,'Raw{}Video'.format(movie_type))
    if os.path.exists(movie_path.replace('.avi','.h5')):
        f = h5py.File(movie_path.replace('.avi','.h5'), 'r')
        frame_intervals = f['frame_intervals'][:]
        return len(frame_intervals) + 1
    else:
        return 0

def get_sync_edge_count(ophys_session_id,movie_type='EyeTracking'):
    wkf = db.get_well_known_files(ophys_session_id)
    sync = vbu.get_sync_data(get_path(wkf,'OphysRigSync'))
    
    if movie_type == 'EyeTracking':
        line = 'cam2_exposure_rising' if 'cam2_exposure_rising' in sync.keys() else 'eye_tracking_rising'
    elif movie_type == 'BehaviorTracking':
        line = 'cam1_exposure_rising' if 'cam1_exposure_rising' in sync.keys() else 'behavior_monitoring_rising'
        
    
    sync_frame_times = sync[line]
    return len(sync_frame_times)

In [34]:
get_sync_edge_count(849597937)

271567

In [11]:
filtered_ophys_experiment_table = dl.get_filtered_ophys_experiment_table()
filtered_ophys_experiment_table

,ophys_session_id,behavior_session_id,container_id,project_code,container_workflow_state,experiment_workflow_state,session_name,session_type,equipment_name,date_of_acquisition,isi_experiment_id,specimen_id,sex,age_in_days,full_genotype,reporter_line,driver_line,imaging_depth,targeted_structure,published_at,super_container_id,cre_line,session_tags,failure_tags,exposure_number,model_outputs_available,location
ophys_experiment_id,,,,,,,,,,,,,,,,,,,,,,,,,,,
953659745,952430817,952554548,1018028339,VisualBehaviorMultiscope,completed,passed,20190923_457841_2imagesA,OPHYS_2_images_A_passive,MESO.1,2019-09-23 08:13:07.627573,858992726,850862430,F,209.0,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Sst-IRES-Cre'],75,VISp,NaN,850862430,Sst-IRES-Cre,NaN,"z_drift_corr_um_diff,z_drift_corr_um_diff,z_dr...",0,False,Sst_VISp_75
953659749,952430817,952554548,1018028354,VisualBehaviorMultiscope,completed,passed,20190923_457841_2imagesA,OPHYS_2_images_A_passive,MESO.1,2019-09-23 08:13:07.627573,858992726,850862430,F,209.0,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Sst-IRES-Cre'],150,VISl,NaN,850862430,Sst-IRES-Cre,NaN,"z_drift_corr_um_diff,z_drift_corr_um_diff,z_dr...",0,False,Sst_VISl_150
953659752,952430817,952554548,1018028357,VisualBehaviorMultiscope,completed,passed,20190923_457841_2imagesA,OPHYS_2_images_A_passive,MESO.1,2019-09-23 08:13:07.627573,858992726,850862430,F,209.0,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Sst-IRES-Cre'],225,VISl,NaN,850862430,Sst-IRES-Cre,NaN,"z_drift_corr_um_diff,z_drift_corr_um_diff,z_dr...",0,False,Sst_VISl_225
953659743,952430817,952554548,1018028345,VisualBehaviorMultiscope,completed,passed,20190923_457841_2imagesA,OPHYS_2_images_A_passive,MESO.1,2019-09-23 08:13:07.627573,858992726,850862430,F,209.0,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Sst-IRES-Cre'],225,VISp,NaN,850862430,Sst-IRES-Cre,NaN,"z_drift_corr_um_diff,z_drift_corr_um_diff,z_dr...",0,False,Sst_VISp_225
958527474,954954402,953982960,1018028339,VisualBehaviorMultiscope,completed,passed,20190924_457841_3imagesA,OPHYS_3_images_A,MESO.1,2019-09-24 16:00:00.000000,858992726,850862430,F,210.0,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Sst-IRES-Cre'],75,VISp,NaN,850862430,Sst-IRES-Cre,NaN,NaN,0,True,Sst_VISp_75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
901559828,900802327,901251663,895259496,VisualBehaviorTask1B,container_qc,passed,20190708_461946_3imagesB,OPHYS_3_images_B,CAM2P.5,2019-07-08 16:09:58.000000,872873989,862671211,F,111.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,['Ai93(TITL-GCaMP6f)'],"['Camk2a-tTA', 'Slc17a7-IRES2-Cre']",175,VISp,NaN,862671211,Slc17a7-IRES2-Cre,NaN,NaN,0,True,Slc17a7_VISp_175
902487182,901759468,902135840,895259496,VisualBehaviorTask1B,container_qc,passed,20190709_461946_2imagesB,OPHYS_2_images_B_passive,CAM2P.5,2019-07-09 15:42:29.000000,872873989,862671211,F,112.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,['Ai93(TITL-GCaMP6f)'],"['Camk2a-tTA', 'Slc17a7-IRES2-Cre']",175,VISp,NaN,862671211,Slc17a7-IRES2-Cre,NaN,NaN,0,False,Slc17a7_VISp_175
903405627,902777219,902871565,895259496,VisualBehaviorTask1B,container_qc,passed,20190710_461946_4imagesA,OPHYS_4_images_A,CAM2P.5,2019-07-10 17:19:50.000000,872873989,862671211,F,113.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,['Ai93(TITL-GCaMP6f)'],"['Camk2a-tTA', 'Slc17a7-IRES2-Cre']",175,VISp,NaN,862671211,Slc17a7-IRES2-Cre,NaN,NaN,0,True,Slc17a7_VISp_175


In [14]:
filtered_experiments_no_eye_tracking = filtered_ophys_experiment_table.reset_index().merge(
    eye_tracking_failures,
    left_on = 'behavior_session_id',
    right_on = 'behavior_session_id',
    how='inner'
)
len(filtered_experiments_no_eye_tracking)

51

In [15]:
filtered_experiments_no_eye_tracking.sort_values(by='ophys_session_id')

,ophys_experiment_id,ophys_session_id,behavior_session_id,container_id,project_code,container_workflow_state,experiment_workflow_state,session_name,session_type,equipment_name,date_of_acquisition,isi_experiment_id,specimen_id,sex,age_in_days,full_genotype,reporter_line,driver_line,imaging_depth,targeted_structure,published_at,super_container_id,cre_line,session_tags,failure_tags,exposure_number,model_outputs_available,location,_id,average_projection,cell_specimen_table,corrected_fluorescence_traces,dff_traces,eye_tracking,licks,max_projection,metadata,motion_correction,ophys_timestamps,rewards,running_data_df,running_speed,segmentation_mask_image,stimulus_presentations,stimulus_templates,stimulus_timestamps,task_parameters,trials,is_ophys,timestamp
34,778644591,778015591,778166125,782536745,VisualBehavior,container_qc,passed,20181113_403491_2imagesA,OPHYS_2_images_A_passive,CAM2P.5,2018-11-13 17:52:17.000000,730814230,722884882,F,165.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,['Ai93(TITL-GCaMP6f)'],"['Camk2a-tTA', 'Slc17a7-IRES2-Cre']",375,VISp,NaN,722884882,Slc17a7-IRES2-Cre,NaN,validate_initial_blank,2,False,Slc17a7_VISp_375,5e901265bdbd55d97f2f5de4,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2020-06-03 10:50:52.352699
48,783927872,783475910,783629024,814796558,VisualBehavior,container_qc,passed,20181120_416369_2P3_VISp_375_1,OPHYS_1_images_A,CAM2P.4,2018-11-20 23:06:02.000000,761346702,756577249,F,105.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,['Ai93(TITL-GCaMP6f)'],"['Camk2a-tTA', 'Slc17a7-IRES2-Cre']",375,VISp,NaN,756577249,Slc17a7-IRES2-Cre,NaN,NaN,0,True,Slc17a7_VISp_375,5e9012c55fc5676828c318cb,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2020-06-03 10:50:12.674696
49,788489531,788418567,788447502,814796558,VisualBehavior,container_qc,passed,20181129_416369_2P3_VISp_375_5,OPHYS_5_images_B_passive,CAM2P.5,2018-11-30 00:15:11.000000,761346702,756577249,F,115.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,['Ai93(TITL-GCaMP6f)'],"['Camk2a-tTA', 'Slc17a7-IRES2-Cre']",375,VISp,NaN,756577249,Slc17a7-IRES2-Cre,NaN,NaN,0,False,Slc17a7_VISp_375,5e901232ec22cc5931035011,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2020-06-03 10:50:19.009001
0,795953296,795625712,795742990,791352433,VisualBehavior,container_qc,passed,20181214_412036_4imagesB,OPHYS_4_images_B,CAM2P.4,2018-12-14 20:17:05.000000,754116317,744911458,F,152.0,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Vip-IRES-Cre'],175,VISp,NaN,744911458,Vip-IRES-Cre,NaN,NaN,0,True,Vip_VISp_175,5e9037596398b8e59ff16e20,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2020-06-03 10:53:32.387562
46,797255551,797078933,797170547,814796612,VisualBehavior,container_qc,passed,20181217_412366_2_images_A_passive,OPHYS_2_images_A_passive,CAM2P.4,2018-12-17 23:37:12.000000,765187424,756674785,F,154.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,['Ai93(TITL-GCaMP6f)'],"['Camk2a-tTA', 'Slc17a7-IRES2-Cre']",375,VISp,NaN,756674785,Slc17a7-IRES2-Cre,NaN,NaN,1,False,Slc17a7_VISp_375,5e903739680b3d7e3e940124,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2020-06-03 10:52:03.878616
40,806456687,805989030,806109931,803517529,VisualBehavior,container_qc,passed,20190109_425493_1imagesA,OPHYS_1_images_A,CAM2P.4,2019-01-09 18:20:00.000000,782229094,772629813,M,106.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,['Ai93(TITL-GCaMP6f)'],"['Camk2a-tTA', 'Slc17a7-IRES2-Cre']",375,VISp,NaN,772629813,Slc17a7-IRES2-Cre,NaN,NaN,0,True,Slc17a7_VISp_375,5e90108a765826c671b756c7,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2020-06-03 10:52:27.624405
41,817267785,817101568,817165094,812858755,VisualBehavior,container_qc,passed,20190201_431023_4imagesB,OPHYS_4_images_B,CAM2P.5,2019-02-01 20:11:18.000000,796874559,789992909,M,99.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,['Ai93(TITL-GCaMP6f)'],"['Camk2a-tTA', 'Slc17a7-IRES2-Cre']",175,VISp,NaN,789992909,Slc17a7-IRES2-Cre,NaN,NaN,0,True,Slc17a7_VISp_175,5e905daad71317d9afcaa284,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2020-06-03 10:5

In [18]:
filtered_experiments_no_eye_tracking['workflow_state'] = filtered_experiments_no_eye_tracking['behavior_session_id'].map(lambda bsid: get_workflow_state(bsid))
for video in ['EyeTracking','BehaviorTracking']:
    filtered_experiments_no_eye_tracking['{}_movie_frame_count'.format(video)] = filtered_experiments_no_eye_tracking['ophys_session_id'].map(lambda osid: get_movie_frame_count(osid,video))
    filtered_experiments_no_eye_tracking['{}_videomon_frame_count'.format(video)] = filtered_experiments_no_eye_tracking['ophys_session_id'].map(lambda osid: get_videomon_frame_count(osid,video))
    filtered_experiments_no_eye_tracking['{}_sync_pulse_count'.format(video)] = filtered_experiments_no_eye_tracking['ophys_session_id'].map(lambda osid: get_sync_edge_count(osid,video))
    

/home/dougo/Code/visual_behavior_analysis/visual_behavior/utilities.py:310: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  timestamp_file = h5py.File(filepath.replace('.avi', '.h5'))


In [19]:
def likely_line_switch(row):
    if (
        row['EyeTracking_movie_frame_count'] == 
        row['EyeTracking_videomon_frame_count'] == 
        row['BehaviorTracking_sync_pulse_count'] and
        row['BehaviorTracking_movie_frame_count'] == 
        row['BehaviorTracking_videomon_frame_count'] == 
        row['EyeTracking_sync_pulse_count']
        ):
        return True
    else:
        return False

In [51]:
def possible_line_switch_eye_would_match(row):
    if (
        row['EyeTracking_movie_frame_count'] !=
        row['EyeTracking_sync_pulse_count'] and
        row['EyeTracking_movie_frame_count'] ==
        row['BehaviorTracking_sync_pulse_count' ]and
        row['likely_line_switch'] == False
        ):
        return True
    else:
        return False

In [50]:
def possible_line_switch_behavior_would_match(row):
    if (
        row['EyeTracking_movie_frame_count'] !=
        row['EyeTracking_sync_pulse_count'] and
        row['BehaviorTracking_movie_frame_count'] ==
        row['EyeTracking_sync_pulse_count'] and
        row['likely_line_switch'] == False
        ):
        return True
    else:
        return False

In [68]:
def missing_eye_video_frames(row):
    if (
        row['EyeTracking_movie_frame_count'] <
        row['EyeTracking_sync_pulse_count'] and
        row['likely_line_switch'] == False and
        row['possible_line_switch_eye_would_match'] == False and
        row['possible_line_switch_behavior_would_match'] == False
        ):
        return True
    else:
        return False
    
def missing_eye_sync_frames(row):
    if (
        row['EyeTracking_movie_frame_count'] >
        row['EyeTracking_sync_pulse_count'] and
        row['likely_line_switch'] == False and
        row['possible_line_switch_eye_would_match'] == False and
        row['possible_line_switch_behavior_would_match'] == False
        ):
        return True
    else:
        return False
    
def appears_fine_but_still_failing(row):
    if row['EyeTracking_sync_pulse_count'] == row['EyeTracking_movie_frame_count']:
        return True
    else:
        return False
    
def missing_vidoemon_log(row):
    if row['EyeTracking_videomon_frame_count'] == 0:
        return True
    else:
        return False
    
def mystery_failure(row):
    if (
        row['likely_line_switch'] == False and
        row['missing_eye_video_frames'] == False and
        row['appears_fine_but_still_failing'] == False and
        row['missing_eye_sync_frames'] == False
        ):
        return True
    else:
        return False

filtered_experiments_no_eye_tracking['eye_movie_sync_mismatch'] = filtered_experiments_no_eye_tracking['EyeTracking_sync_pulse_count'] - filtered_experiments_no_eye_tracking['EyeTracking_movie_frame_count']
filtered_experiments_no_eye_tracking['likely_line_switch'] = filtered_experiments_no_eye_tracking.apply(likely_line_switch, axis=1)
filtered_experiments_no_eye_tracking['possible_line_switch_eye_would_match'] = filtered_experiments_no_eye_tracking.apply(possible_line_switch_eye_would_match, axis=1)
filtered_experiments_no_eye_tracking['possible_line_switch_behavior_would_match'] = filtered_experiments_no_eye_tracking.apply(possible_line_switch_behavior_would_match, axis=1)
filtered_experiments_no_eye_tracking['missing_eye_video_frames'] = filtered_experiments_no_eye_tracking.apply(missing_eye_video_frames, axis=1)
filtered_experiments_no_eye_tracking['missing_eye_sync_frames'] = filtered_experiments_no_eye_tracking.apply(missing_eye_sync_frames, axis=1)
filtered_experiments_no_eye_tracking['appears_fine_but_still_failing'] = filtered_experiments_no_eye_tracking.apply(appears_fine_but_still_failing, axis=1)
filtered_experiments_no_eye_tracking['missing_vidoemon_log'] = filtered_experiments_no_eye_tracking.apply(missing_vidoemon_log, axis=1)
filtered_experiments_no_eye_tracking['mystery_failure'] = filtered_experiments_no_eye_tracking.apply(mystery_failure, axis=1)

In [69]:
cols = [
    'behavior_session_id',
    'ophys_session_id',
    'ophys_experiment_id',
    'project_code',
    'workflow_state',
    'equipment_name',
    'date_of_acquisition'
]
cols = cols + [
    'EyeTracking_movie_frame_count',
    'EyeTracking_videomon_frame_count',
    'EyeTracking_sync_pulse_count',
    'BehaviorTracking_movie_frame_count',
    'BehaviorTracking_videomon_frame_count',
    'BehaviorTracking_sync_pulse_count',
    'eye_movie_sync_mismatch'
]
error_types = [
    'likely_line_switch',
    'possible_line_switch_eye_would_match',
    'possible_line_switch_behavior_would_match',
    'missing_eye_video_frames',
    'missing_eye_sync_frames',
    'appears_fine_but_still_failing',
    'missing_vidoemon_log',
    'mystery_failure'
]
cols = cols + error_types
filtered_experiments_no_eye_tracking[cols].sort_values(by='date_of_acquisition').reset_index(drop=True)

,behavior_session_id,ophys_session_id,ophys_experiment_id,project_code,workflow_state,equipment_name,date_of_acquisition,EyeTracking_movie_frame_count,EyeTracking_videomon_frame_count,EyeTracking_sync_pulse_count,BehaviorTracking_movie_frame_count,BehaviorTracking_videomon_frame_count,BehaviorTracking_sync_pulse_count,eye_movie_sync_mismatch,likely_line_switch,possible_line_switch_eye_would_match,possible_line_switch_behavior_would_match,missing_eye_video_frames,missing_eye_sync_frames,appears_fine_but_still_failing,missing_vidoemon_log,mystery_failure
0,778166125,778015591,778644591,VisualBehavior,passed,CAM2P.5,2018-11-13 17:52:17.000000,144999,145003,145003,144976,144976,144976,4,False,False,False,True,False,False,False,False
1,783629024,783475910,783927872,VisualBehavior,passed,CAM2P.4,2018-11-20 23:06:02.000000,144868,144873,144997,144997,144997,144873,129,False,False,True,False,False,False,False,True
2,788447502,788418567,788489531,VisualBehavior,passed,CAM2P.5,2018-11-30 00:15:11.000000,145025,145035,145035,145027,145027,145027,10,False,False,False,True,False,False,False,False
3,795742990,795625712,795953296,VisualBehavior,passed,CAM2P.4,2018-12-14 20:17:05.000000,135928,135931,135924,135924,135924,135931,-4,False,False,True,False,False,False,False,True
4,797170547,797078933,797255551,VisualBehavior,passed,CAM2P.4,2018-12-17 23:37:12.000000,135758,135758,135878,135878,135878,135758,120,True,False,False,False,False,False,False,False
5,806109931,805989030,806456687,VisualBehavior,passed,CAM2P.4,2019-01-09 18:20:00.000000,135917,135920,135915,135915,135915,135920,-2,False,False,True,False,False,False,False,True
6,817165094,817101568,817267785,VisualBehavior,passed,CAM2P.5,2019-02-01 20:11:18.000000,135917,135918,135918,135910,135910,135910,1,False,False,False,True,False,False,False,False
7,822249041,822081744,822641265,VisualBehavior,passed,CAM2P.3,2019-02-13 16:22:31.000000,272306,272311,272311,136186,136186,136186,5,False,False,False,True,False,False,False,False
8,832981154,832881662,833631914,VisualBehavior,passed,CAM2P.4,2019-03-06 15:34:35.000000,135987,135992,135972,135972,135972,135992,-15,False,False,True,False,False,False,False,True
9,837108110,836962545,837296345,VisualBehavior,passed,CAM2P.4,2019-03-14 14:32:50.000000,135835,135851,135843,135843,135843,135851,8,False,False,True,False,False,False,False,True


In [35]:
filtered_experiments_no_eye_tracking.to_csv('/allen/programs/braintv/workgroups/nc-ophys/visual_behavior/2020.07.29_sdk_issue_1545_log.csv', index=False)

In [36]:
filtered_experiments_no_eye_tracking[error_types].sum(axis=0)

likely_line_switch                 1
missing_eye_video_frames          28
missing_eye_sync_frames           17
appears_fine_but_still_failing     5
missing_vidoemon_log              23
mystery_failure                    3
dtype: int64

In [37]:
filtered_experiments_no_eye_tracking[error_types].sum(axis=1).reset_index(drop=True)

0     2
1     2
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
11    2
12    2
13    2
14    2
15    2
16    2
17    2
18    2
19    2
20    2
21    2
22    2
23    2
24    2
25    2
26    2
27    2
28    2
29    2
30    2
31    2
32    2
33    2
34    1
35    1
36    1
37    1
38    1
39    1
40    2
41    1
42    1
43    1
44    1
45    1
46    1
47    1
48    1
49    1
50    1
dtype: int64

## error type: likely line switch
eye video count matches behavior sync count; behavior video count matches eye sync count

In [41]:
cols_to_display = [
    'ophys_session_id',
    'ophys_experiment_id',
    'project_code',
    'workflow_state',
    'equipment_name',
    'date_of_acquisition'
]
cols_to_display  = cols_to_display + [
    'EyeTracking_movie_frame_count',
    'EyeTracking_sync_pulse_count',
    'BehaviorTracking_movie_frame_count',
    'BehaviorTracking_sync_pulse_count',
    'eye_movie_sync_mismatch'
]
filtered_experiments_no_eye_tracking.query('likely_line_switch')[cols_to_display].sort_values(by='date_of_acquisition').reset_index(drop=True)

,ophys_session_id,ophys_experiment_id,project_code,workflow_state,equipment_name,date_of_acquisition,EyeTracking_movie_frame_count,EyeTracking_sync_pulse_count,BehaviorTracking_movie_frame_count,BehaviorTracking_sync_pulse_count,eye_movie_sync_mismatch
0,797078933,797255551,VisualBehavior,passed,CAM2P.4,2018-12-17 23:37:12.000000,135758,135878,135878,135758,120


## error type: likely line switch w/ eye mismatch
behavior video count matches eye sync count

In [58]:
cols_to_display = [
    'ophys_session_id',
    'ophys_experiment_id',
    'project_code',
    'workflow_state',
    'equipment_name',
    'date_of_acquisition'
]
cols_to_display  = cols_to_display + [
    'EyeTracking_movie_frame_count',
    'EyeTracking_sync_pulse_count',
    'BehaviorTracking_movie_frame_count',
    'BehaviorTracking_sync_pulse_count',
    'eye_movie_sync_mismatch',
]
filtered_experiments_no_eye_tracking.query('possible_line_switch_eye_would_match or possible_line_switch_behavior_would_match')[cols_to_display].sort_values(by='date_of_acquisition').reset_index(drop=True)


,ophys_session_id,ophys_experiment_id,project_code,workflow_state,equipment_name,date_of_acquisition,EyeTracking_movie_frame_count,EyeTracking_sync_pulse_count,BehaviorTracking_movie_frame_count,BehaviorTracking_sync_pulse_count,eye_movie_sync_mismatch
0,783475910,783927872,VisualBehavior,passed,CAM2P.4,2018-11-20 23:06:02.000000,144868,144997,144997,144873,129
1,795625712,795953296,VisualBehavior,passed,CAM2P.4,2018-12-14 20:17:05.000000,135928,135924,135924,135931,-4
2,805989030,806456687,VisualBehavior,passed,CAM2P.4,2019-01-09 18:20:00.000000,135917,135915,135915,135920,-2
3,832881662,833631914,VisualBehavior,passed,CAM2P.4,2019-03-06 15:34:35.000000,135987,135972,135972,135992,-15
4,836962545,837296345,VisualBehavior,passed,CAM2P.4,2019-03-14 14:32:50.000000,135835,135843,135843,135851,8


In [60]:
filtered_experiments_no_eye_tracking.query('possible_line_switch_eye_would_match or possible_line_switch_behavior_would_match')['ophys_experiment_id'].to_list()

[795953296, 833631914, 806456687, 837296345, 783927872]

## error type: missing eye frames
the eye tracking movie count is less than the eye tracking sync count

In [73]:
cols_to_display = [
    'ophys_session_id',
    'ophys_experiment_id',
    'project_code',
    'workflow_state',
    'equipment_name',
    'date_of_acquisition'
]
cols_to_display  = cols_to_display + [
    'EyeTracking_movie_frame_count',
    'EyeTracking_sync_pulse_count',
    'BehaviorTracking_movie_frame_count',
    'BehaviorTracking_sync_pulse_count',
    'eye_movie_sync_mismatch',
]
filtered_experiments_no_eye_tracking.query('missing_eye_video_frames')[cols_to_display].sort_values(by='date_of_acquisition').drop_duplicates().reset_index(drop=True)


,ophys_session_id,ophys_experiment_id,project_code,workflow_state,equipment_name,date_of_acquisition,EyeTracking_movie_frame_count,EyeTracking_sync_pulse_count,BehaviorTracking_movie_frame_count,BehaviorTracking_sync_pulse_count,eye_movie_sync_mismatch
0,778015591,778644591,VisualBehavior,passed,CAM2P.5,2018-11-13 17:52:17.000000,144999,145003,144976,144976,4
1,788418567,788489531,VisualBehavior,passed,CAM2P.5,2018-11-30 00:15:11.000000,145025,145035,145027,145027,10
2,817101568,817267785,VisualBehavior,passed,CAM2P.5,2019-02-01 20:11:18.000000,135917,135918,135910,135910,1
3,822081744,822641265,VisualBehavior,passed,CAM2P.3,2019-02-13 16:22:31.000000,272306,272311,136186,136186,5
4,873653940,873968820,VisualBehavior,passed,CAM2P.4,2019-05-22 19:09:19.000000,136012,136017,136009,136009,5
5,877946125,878363088,VisualBehavior,passed,CAM2P.3,2019-05-30 17:50:27.000000,270860,270867,135742,135742,7
6,882351065,882520593,VisualBehavior,passed,CAM2P.4,2019-06-06 20:01:28.000000,135873,135876,135849,135849,3
7,910985315,911146705,VisualBehaviorTask1B,passed,CAM2P.5,2019-07-23 20:31:30.000000,136115,136119,136096,136096,4
8,931326814,932372699,VisualBehaviorMultiscope,passed,MESO.1,2019-08-26 08:26:26.815263,135814,135820,135809,135809,6
9,931326814,932372701,VisualBehaviorMultiscope,passed,MESO.1,2019-08-26 08:26:26.815263,135814,135820,135809,135809,6


In [75]:
filtered_experiments_no_eye_tracking.query('missing_eye_video_frames')['ophys_experiment_id'].drop_duplicates().to_list()

[878363088,
 873968820,
 939327156,
 940354166,
 934476801,
 938001540,
 940354181,
 932372711,
 932372707,
 932372705,
 932372701,
 932372699,
 778644591,
 972701564,
 979668410,
 822641265,
 817267785,
 963394065,
 1004889552,
 882520593,
 911146705,
 788489531]

## error type: missing eye sync pulses
There are more video frames than sync pulses

In [76]:
cols_to_display = [
    'ophys_session_id',
    'ophys_experiment_id',
    'project_code',
    'workflow_state',
    'equipment_name',
    'date_of_acquisition'
]
cols_to_display  = cols_to_display + [
    'EyeTracking_movie_frame_count',
    'EyeTracking_sync_pulse_count',
    'BehaviorTracking_movie_frame_count',
    'BehaviorTracking_sync_pulse_count',
    'eye_movie_sync_mismatch',
]
filtered_experiments_no_eye_tracking.query('missing_eye_sync_frames')[cols_to_display].sort_values(by='date_of_acquisition').drop_duplicates().reset_index(drop=True)

,ophys_session_id,ophys_experiment_id,project_code,workflow_state,equipment_name,date_of_acquisition,EyeTracking_movie_frame_count,EyeTracking_sync_pulse_count,BehaviorTracking_movie_frame_count,BehaviorTracking_sync_pulse_count,eye_movie_sync_mismatch
0,945124131,945586450,VisualBehaviorMultiscope,passed,MESO.1,2019-09-12 14:38:17.192559,135725,135646,135755,135676,-79
1,945124131,945586458,VisualBehaviorMultiscope,passed,MESO.1,2019-09-12 14:38:17.192559,135725,135646,135755,135676,-79
2,945124131,945586456,VisualBehaviorMultiscope,passed,MESO.1,2019-09-12 14:38:17.192559,135725,135646,135755,135676,-79
3,945124131,945586454,VisualBehaviorMultiscope,passed,MESO.1,2019-09-12 14:38:17.192559,135725,135646,135755,135676,-79
4,945124131,945586452,VisualBehaviorMultiscope,passed,MESO.1,2019-09-12 14:38:17.192559,135725,135646,135755,135676,-79
5,945124131,945586448,VisualBehaviorMultiscope,passed,MESO.1,2019-09-12 14:38:17.192559,135725,135646,135755,135676,-79
6,945124131,945586446,VisualBehaviorMultiscope,passed,MESO.1,2019-09-12 14:38:17.192559,135725,135646,135755,135676,-79
7,945124131,945586444,VisualBehaviorMultiscope,passed,MESO.1,2019-09-12 14:38:17.192559,135725,135646,135755,135676,-79
8,1012618881,1012771667,VisualBehaviorMultiscope4areasx2d,passed,MESO.1,2020-03-05 11:30:19.493894,136935,136903,136926,136893,-32
9,1012618881,1012771673,VisualBehaviorMultiscope4areasx2d,passed,MESO.1,2020-03-05 11:30:19.493894,136935,136903,136926,136893,-32


In [77]:
filtered_experiments_no_eye_tracking.query('missing_eye_sync_frames')[cols_to_display].sort_values(by='date_of_acquisition').drop_duplicates().reset_index(drop=True)['ophys_experiment_id'].to_list()

[945586450,
 945586458,
 945586456,
 945586454,
 945586452,
 945586448,
 945586446,
 945586444,
 1012771667,
 1012771673,
 1012771672,
 1012771678,
 1012771669,
 1012771670]

## error type: sync mismatch in SDK but not sync file
when loading with the SDK, a sync mismatch is reported. But the sync pulse count in the sync file appears fine


In [78]:
cols_to_display = [
    'ophys_session_id',
    'ophys_experiment_id',
    'project_code',
    'workflow_state',
    'equipment_name',
    'date_of_acquisition'
]
cols_to_display  = cols_to_display + [
    'EyeTracking_movie_frame_count',
    'EyeTracking_sync_pulse_count',
    'BehaviorTracking_movie_frame_count',
    'BehaviorTracking_sync_pulse_count',
    'eye_movie_sync_mismatch',
]
filtered_experiments_no_eye_tracking.query('appears_fine_but_still_failing')[cols_to_display].sort_values(by='date_of_acquisition').drop_duplicates().reset_index(drop=True)

,ophys_session_id,ophys_experiment_id,project_code,workflow_state,equipment_name,date_of_acquisition,EyeTracking_movie_frame_count,EyeTracking_sync_pulse_count,BehaviorTracking_movie_frame_count,BehaviorTracking_sync_pulse_count,eye_movie_sync_mismatch
0,849597937,850479305,VisualBehavior,passed,CAM2P.3,2019-04-11 18:56:20.000000,271567,271567,135794,135794,0
1,877907546,878358326,VisualBehavior,passed,CAM2P.4,2019-05-30 17:29:35.000000,136042,136042,136001,136001,0
2,968900812,969809655,VisualBehaviorTask1B,passed,CAM2P.4,2019-10-17 15:02:31.000000,135586,135586,135567,135567,0
3,992675762,992868972,VisualBehaviorTask1B,passed,CAM2P.4,2019-12-06 16:28:04.000000,135786,135786,135756,135756,0
4,994689422,994791582,VisualBehaviorTask1B,passed,CAM2P.4,2019-12-16 17:00:21.000000,135697,135697,135668,135668,0


In [79]:
filtered_experiments_no_eye_tracking.query('appears_fine_but_still_failing')[cols_to_display].sort_values(by='date_of_acquisition').drop_duplicates().reset_index(drop=True)['ophys_experiment_id'].to_list()

[850479305, 878358326, 969809655, 992868972, 994791582]

In [80]:
from allensdk.brain_observatory.behavior.behavior_ophys_session import BehaviorOphysSession
oeid = 850479305
session = BehaviorOphysSession.from_lims(oeid)
session.eye_tracking

/home/dougo/Code/AllenSDK/allensdk/brain_observatory/sync_dataset.py:109: UserWarning: The loaded sync file contains the following deprecated line label keys: {'cam1_exposure', 'cam2_exposure'}. Consider updating the sync file line labels.
  self._check_line_labels()


RuntimeError: Error! The number of sync file frame times (124540 does not match the number of eye tracking frames (271567)!

In [83]:
filtered_ophys_experiment_table.query('ophys_session_id == 945124131')

,ophys_session_id,behavior_session_id,container_id,project_code,container_workflow_state,experiment_workflow_state,session_name,session_type,equipment_name,date_of_acquisition,isi_experiment_id,specimen_id,sex,age_in_days,full_genotype,reporter_line,driver_line,imaging_depth,targeted_structure,published_at,super_container_id,cre_line,session_tags,failure_tags,exposure_number,model_outputs_available,location
ophys_experiment_id,,,,,,,,,,,,,,,,,,,,,,,,,,,
945586450,945124131,945277725,1018028396,VisualBehaviorMultiscope,completed,passed,20190912_453989_Ophys4,OPHYS_4_images_B,MESO.1,2019-09-12 14:38:17.192559,848540715,840378937,M,216.0,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Vip-IRES-Cre'],225,VISp,NaN,840378937,Vip-IRES-Cre,NaN,NaN,0,False,Vip_VISp_225
945586458,945124131,945277725,1018028408,VisualBehaviorMultiscope,completed,passed,20190912_453989_Ophys4,OPHYS_4_images_B,MESO.1,2019-09-12 14:38:17.192559,848540715,840378937,M,216.0,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Vip-IRES-Cre'],225,VISl,NaN,840378937,Vip-IRES-Cre,NaN,NaN,0,False,Vip_VISl_225
945586456,945124131,945277725,1018028405,VisualBehaviorMultiscope,completed,passed,20190912_453989_Ophys4,OPHYS_4_images_B,MESO.1,2019-09-12 14:38:17.192559,848540715,840378937,M,216.0,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Vip-IRES-Cre'],150,VISl,NaN,840378937,Vip-IRES-Cre,NaN,NaN,0,False,Vip_VISl_150
945586454,945124131,945277725,1018028411,VisualBehaviorMultiscope,completed,passed,20190912_453989_Ophys4,OPHYS_4_images_B,MESO.1,2019-09-12 14:38:17.192559,848540715,840378937,M,216.0,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Vip-IRES-Cre'],300,VISl,NaN,840378937,Vip-IRES-Cre,NaN,NaN,0,False,Vip_VISl_300
945586452,945124131,945277725,1018028402,VisualBehaviorMultiscope,completed,passed,20190912_453989_Ophys4,OPHYS_4_images_B,MESO.1,2019-09-12 14:38:17.192559,848540715,840378937,M,216.0,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Vip-IRES-Cre'],75,VISl,NaN,840378937,Vip-IRES-Cre,NaN,NaN,0,False,Vip_VISl_75
945586448,945124131,945277725,1018028393,VisualBehaviorMultiscope,completed,passed,20190912_453989_Ophys4,OPHYS_4_images_B,MESO.1,2019-09-12 14:38:17.192559,848540715,840378937,M,216.0,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Vip-IRES-Cre'],150,VISp,NaN,840378937,Vip-IRES-Cre,NaN,NaN,0,False,Vip_VISp_150
945586446,945124131,945277725,1018028399,VisualBehaviorMultiscope,completed,passed,20190912_453989_Ophys4,OPHYS_4_images_B,MESO.1,2019-09-12 14:38:17.192559,848540715,840378937,M,216.0,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Vip-IRES-Cre'],300,VISp,NaN,840378937,Vip-IRES-Cre,NaN,NaN,0,False,Vip_VISp_300
945586444,945124131,945277725,1018028390,VisualBehaviorMultiscope,completed,passed,20190912_453989_Ophys4,OPHYS_4_images_B,MESO.1,2019-09-12 14:38:17.192559,848540715,840378937,M,216.0,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Vip-IRES-Cre'],75,VISp,NaN,840378937,Vip-IRES-Cre,NaN,NaN,0,False,Vip_VISp_75


In [84]:
filtered_ophys_experiment_table.query('ophys_session_id == 1012618881')

,ophys_session_id,behavior_session_id,container_id,project_code,container_workflow_state,experiment_workflow_state,session_name,session_type,equipment_name,date_of_acquisition,isi_experiment_id,specimen_id,sex,age_in_days,full_genotype,reporter_line,driver_line,imaging_depth,targeted_structure,published_at,super_container_id,cre_line,session_tags,failure_tags,exposure_number,model_outputs_available,location
ophys_experiment_id,,,,,,,,,,,,,,,,,,,,,,,,,,,
1012771667,1012618881,1012667846,1018027962,VisualBehaviorMultiscope4areasx2d,holding,passed,20200305_489065_Ophys6,OPHYS_6_images_H,MESO.1,2020-03-05 11:30:19.493894,972289328,958769031,F,214.0,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Vip-IRES-Cre'],173,VISp,NaN,958769031,Vip-IRES-Cre,NaN,NaN,0,False,Vip_VISp_173
1012771673,1012618881,1012667846,1018027974,VisualBehaviorMultiscope4areasx2d,holding,passed,20200305_489065_Ophys6,OPHYS_6_images_H,MESO.1,2020-03-05 11:30:19.493894,972289328,958769031,F,214.0,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Vip-IRES-Cre'],181,VISal,NaN,958769031,Vip-IRES-Cre,NaN,NaN,0,False,Vip_VISal_181
1012771672,1012618881,1012667846,1018027971,VisualBehaviorMultiscope4areasx2d,holding,passed,20200305_489065_Ophys6,OPHYS_6_images_H,MESO.1,2020-03-05 11:30:19.493894,972289328,958769031,F,214.0,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Vip-IRES-Cre'],269,VISl,NaN,958769031,Vip-IRES-Cre,NaN,NaN,0,False,Vip_VISl_269
1012771678,1012618881,1012667846,1018027981,VisualBehaviorMultiscope4areasx2d,holding,passed,20200305_489065_Ophys6,OPHYS_6_images_H,MESO.1,2020-03-05 11:30:19.493894,972289328,958769031,F,214.0,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Vip-IRES-Cre'],167,VISam,NaN,958769031,Vip-IRES-Cre,NaN,NaN,0,False,Vip_VISam_167
1012771669,1012618881,1012667846,1018027965,VisualBehaviorMultiscope4areasx2d,holding,passed,20200305_489065_Ophys6,OPHYS_6_images_H,MESO.1,2020-03-05 11:30:19.493894,972289328,958769031,F,214.0,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Vip-IRES-Cre'],269,VISp,NaN,958769031,Vip-IRES-Cre,NaN,NaN,0,False,Vip_VISp_269
1012771670,1012618881,1012667846,1018027968,VisualBehaviorMultiscope4areasx2d,holding,passed,20200305_489065_Ophys6,OPHYS_6_images_H,MESO.1,2020-03-05 11:30:19.493894,972289328,958769031,F,214.0,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Vip-IRES-Cre'],179,VISl,NaN,958769031,Vip-IRES-Cre,NaN,NaN,0,False,Vip_VISl_179
